In [1]:
# Import libraries 
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Import the code for fetching GA Data
from gaData import *

In [5]:
######### Configure the inputs ##################
### Replace with your own view_id
gaView = "xxxxxxxx"
### Set the date range for your data
start_date='2018-01-01'
end_date='2018-11-30'
### Replace with your own segmentId you found in Query Explorer
segments = [{"segmentId": "gaid::xxxxxxxxxxxxxx"}]
### To get data, we must have one metrics, here we use sessions
metrics=[{"expression": "ga:sessions"}]
### Replace with your own custom dimention ID and include segment also
dimensions=[{"name": "ga:dimensionXX"},{"name": "ga:segment"}]
### Your data will be grouped by the dimensions you use 
group_by = ["ga:dimensionXX","ga:segment"]
### Set your client secret path where you put your client credentials 
CLIENT_SECRETS_PATH = 'key/client_secret.json'

In [6]:
# Initiate the analytics client
analytics = initialize_analyticsreporting(CLIENT_SECRETS_PATH)

In [7]:
######### Run the program to get data ##################
start_time = time.time()
data = return_ga_data(analytics,
                      start_date= start_date,
                      end_date=end_date,
                      view_id=gaView,
                      metrics=metrics,
                      dimensions=dimensions,
                      segments = segments,
                      group_by = group_by)
print("Running Time --- %s seconds ---" % (time.time() - start_time))
data.head()

Running Time --- 1559.456502199173 seconds ---


,ga:dimension42,ga:segment,ga:sessions
0,1001033209.1481225329,phoneNumber,2.0
1,1001090294.1528794762,phoneNumber,1.0
2,1007582195.1522236043,phoneNumber,3.0
3,1010110771.1512153214,phoneNumber,1.0
4,101499865.1532375076,phoneNumber,1.0


In [2]:
# Import libraries for connect to google spread sheet
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g

In [12]:
# Specify we are using client IDs to delete users
data['typeId'] = 'CLIENT_ID'
# Select the columns we need to upload to the Google Sheet
df = data[['ga:dimension42','typeId']]

In [13]:
df.head()

,ga:dimension42,typeId
0,1.001033e+09,CLIENT_ID
1,1.001090e+09,CLIENT_ID
2,1.007582e+09,CLIENT_ID
3,1.010111e+09,CLIENT_ID
4,1.014999e+08,CLIENT_ID


In [16]:
# Configure the connection 
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('key/service_account_GS.json', scope)
gc = gspread.authorize(credentials)
#### The sprad sheet ID, which can be taken from the link to the sheet
spreadsheet_key = '1uHQgUSElHaKSKATOYUxZjwnNx4NuR0U0XDjieIkTToM'

In [18]:
# Set the sheet name you want to upload data to and the start cell where the upload data begins 
wks_name = 'Main'
cell_of_start_df = 'A5'
# upload the dataframe of the clients we want to delete
d2g.upload(df,
           spreadsheet_key,
           wks_name,
           credentials=credentials,
           col_names=False,
           row_names=False,
           start_cell = cell_of_start_df,
           clean=False)
print ('The sheet is updated successfully')

The sheet is updated successfully
